In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm

Lecture 3 Slide 24/34: 

Solve the following PDE:

$$ \frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2\frac{\partial^2 V}{\partial x^2} = 0 $$
$$ V(x,T) = x^2 $$
$$ V(x,t) = ? $$

Note:

$$ dX(s) = 0*dt + \sigma*dW(s)  $$
where r = 0 
$$ X(t + \Delta t) = X(t) + \sigma \sqrt{\Delta t} \cdot Z $$
$$ X(t) = x $$
$$ X \sim N(x,\sigma^2T) $$

In [2]:
### Simulation approach. Slide 26/34. (Simulation of Montecarlo paths for the stochastic process X, calculating V and then taking expectation)
def GeneratePaths(NoOfPaths,NoOfSteps,x,T,sigma):
    Z = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    time = np.zeros([NoOfSteps+1])

    X[:,0] = x

    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):
        # making sure that samples from normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])

        X[:,i+1] = X[:,i] + sigma * np.power(dt, 0.5)*Z[:,i]
        time[i+1] = time[i] +dt

    paths = {"time":time, "X":X}
    return paths

def MainCode():
    NoOfPaths = 1000
    NoOfSteps = 10000
    x         = 3
    sigma     = 0.2
    T         = 10
    r = 0
    
    # Monte Carlo Paths

    paths    = GeneratePaths(NoOfPaths,NoOfSteps,x,T,sigma)
    X        = paths["X"]
    print("The mean of X is: " + str(np.mean(X[:,-1])) + " with the expected value of: ", str(x))
    print("The std dev of X is: " + str(np.std(X[:,-1])) + " with the expected value of: ", sigma*np.sqrt(T))
    time= paths["time"]
    V = np.square(X[:,-1])
    EV = np.exp(-r*T) * np.mean(V)
    print("The value of V(x,t) is: " + str(EV) + " with the expected value of: ", np.square(sigma)*T + np.square(x))

In [3]:
MainCode()

The mean of X is: 3.0 with the expected value of:  3
The std dev of X is: 0.6246168329540882 with the expected value of:  0.632455532033676
The value of V(x,t) is: 9.390146188009595 with the expected value of:  9.4


In [4]:
### Integral approach. Slide 26/34.
### The solution of the integrand is sigma^2 * T + x^2

# Define the normal distribution PDF with mean x and variance sigma^2 * t
def normal_pdf(z):
    mu = x
    variance = sigma**2 * T
    return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-(z-mu)**2 / (2 * variance))

# Define the integrand: x^2 * PDF of normal distribution with mean x and variance sigma^2 * t
def integrand(z):
    return z**2 * normal_pdf(z)

x         = 3
sigma     = 0.2
T         = 10
r = 0

# Compute the integral from -inf to +inf
V, _ = quad(integrand, -np.inf, np.inf)    
EV = np.exp(-r*T) * V
print("The value of V(x,t) is: " + str(EV) + " with the expected value of: ", sigma**2 * T + x**2 )

The value of V(x,t) is: 9.399999999999958 with the expected value of:  9.4


Taking another example fron Slide 31/34 where V = max(S-K,0): 

$$ dS(t) = r S(t)dt + \sigma S(t) dW^Q(t)$$

In [5]:
### Simulation approach. Slide 26/34.
def GeneratePathsGBM(NoOfPaths,NoOfSteps,T,r,sigma,S_0):

    # Fixing random seed
    np.random.seed(1)

    Z = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    X = np.zeros([NoOfPaths, NoOfSteps+1])
    S = np.zeros([NoOfPaths, NoOfSteps+1])
    time = np.zeros([NoOfSteps+1])

    X[:,0] = np.log(S_0)

    dt = T / float(NoOfSteps)
    for i in range(0,NoOfSteps):
        # making sure that samples from normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])

        X[:,i+1] = X[:,i] + (r - 0.5 * sigma **2 ) * dt + sigma * np.power(dt, 0.5)*Z[:,i]
        time[i+1] = time[i] +dt

    #Compute exponent of ABM
    S = np.exp(X)
    paths = {"time":time,"X":X,"S":S}
    return paths

def black_scholes_call(S, K, T, r, sigma):
    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate call price
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def mainCalculation():
    NoOfPaths = 500
    NoOfSteps = 10000
    T = 1
    r = 0.05
    sigma = 0.4
    S_0 = 100
    K = 95

    Paths = GeneratePathsGBM(NoOfPaths,NoOfSteps,T,r,sigma,S_0)
    timeGrid = Paths["time"]
    X = Paths["X"]
    S = Paths["S"]

    
    V = S[:,-1] - K
    V[V < 0] = 0
    EV = np.exp(-r*T) * np.mean(V)
    print("The value of V(x,t) is: " + str(EV) + " with the expected Black-Scholes price of: ", black_scholes_call(S_0, K, T, r, sigma))
mainCalculation()

The value of V(x,t) is: 21.892032074316216 with the expected Black-Scholes price of:  20.379349629711726


In [6]:
### Integral approach. Slide 26/34.

# Define the normal distribution PDF with mean x and variance sigma^2 * t
def normal_pdf(z):
    mu = ( r - sigma**2 / 2 ) * T
    variance = sigma**2 * T
    return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-(z-mu)**2 / (2 * variance))

# Define the integrand: x^2 * PDF of normal distribution with mean x and variance sigma^2 * t
def integrand(z):
    return (S_0 * np.exp(z) - K) * normal_pdf(z)

T = 1
r = 0.05
sigma = 0.4
S_0 = 100
K = 95

# Compute the integral from -inf to +inf. However, this throw an error due to instability caused by roundoff errors so we capped the limit to 50.
# V, _ = quad(integrand, np.log(K/S_0), np.inf)    
V, _ = quad(integrand, np.log(K / S_0), 50, epsabs=1e-6, epsrel=1e-6)
EV = np.exp(-r*T) * V
print("The value of V(x,t) is: " + str(EV) + " with the expected value of: ", black_scholes_call(S_0, K, T, r, sigma))

The value of V(x,t) is: 20.379349629711708 with the expected value of:  20.379349629711726
